# Toolkit Prerequisites and Initial Setup
Most of the notebooks in the Azure SQL Hybrid Cloud Toolkit leverage either PowerShell or Python for their script functionality.  This notebook page will do the local machine configuration necessary to execute the various notebooks successfully.  

## PowerShell Setup

The modules required by the toolkit are available on the PowerShell Gallery.  General information on the PowerShell Gallery can be found at https://www.powershellgallery.com/.  The setup code in this notebook will create a temporary Gallery registration to install required PowerShell modules.  If you do not have direct access to the PowerShell Gallery or have not ever done the installation of the PowerShellGet module then please see the directions here: https://docs.microsoft.com/en-us/powershell/scripting/gallery/installing-psget?view=powershell-7.  The initial setup for the PowerShellGet module requires elevated permissions and is recommended to be done in a separate PowerShell command prompt.

## PS Version and Modules
Many notebooks use a <a href="https://youtu.be/BhbiAINQBYE">PowerShell Notebook kernel</a> (YouTube video) and requires PS v5.1 or greater.

In [ ]:
# Check that PowerShell >= v5.1 and Azure Command Line Interface (Az CLI) are installed
if ($PSVersionTable.PSVersion.Major -lt 5)
{
    Write-Warning "Some notebooks require a greater version of PowerShell."
}
else {
    $psVersion = $PSVersionTable.PSVersion.ToString()
    Write-Output "Found PS version... $psVersion"
}

### Required Modules
|Module|Description|
|---|---|
|dbatools| Provides high-level cmdlets that perform SQL management operations.  These CMDLETs have extensive validation tests and configuration options.|
|Az.Accounts| Provides cmdlets to configure subscription and account settings|
|Az.Resources| Provides cmdlets to access and provision Microsoft Azure resources|
|Az.Storage| Provides cmdlets to simplify configuration and access to Azure storage resources|
|Az.Network| Networking cmdlets for Azure Resource Manager
|Az.Compute| Compute service cmdlets
|AnyBox| Provides GUI input box support (Windows only)|
|SqlServer| Provides cmdlets to automate database development and server administration, as well as both multidimensional and tabular cube processing |

If the above modules have not already been installed the following code will install them only for current user scope.

*Note: this temporary gallery registration is setup as trusted to enable installation of modules without prompting. After installing the modules, the gallery registration will be removed.*

In [ ]:
# need to check if nuget package provider is installed
$nuget=(Get-PackageProvider -Name NuGet)

if (!$nuget)
{
    Get-PackageProvider -Name NuGet -ForceBootstrap
    Import-PackageProvider -Name NuGet -RequiredVersion 2.8.5.201
}

#temporary reposistory registration
Register-PSRepository -Name PSGalleryTemp -SourceLocation https://www.powershellgallery.com/api/v2/ -PublishLocation https://www.powershellgallery.com/api/v2/package/ -ScriptSourceLocation https://www.powershellgallery.com/api/v2/items/psscript/ -ScriptPublishLocation https://www.powershellgallery.com/api/v2/package/ -InstallationPolicy Trusted -PackageManagementProvider NuGet

#required modules for this notebook
$requiredModules = @("dbatools", "Az.Accounts", "Az.Resources", "Az.Storage", "Az.Network", "Az.Compute", "AnyBox","SqlServer")

#check each module and install if necessary
foreach ($moduleName in $requiredModules) {
    if (!(Get-Module -ListAvailable -Name $moduleName)) {
        Install-Module -Name $moduleName -Scope CurrentUser -AllowClobber -Repository PSGalleryTemp
    }
} 

#remove temporary gallery registration
Unregister-PSRepository -Name PSGalleryTemp

foreach ($moduleName in $requiredModules) {
       $module = Get-InstalledModule $moduleName
    if (!($module)) {
          Write-Warning "The specified module is not installed...$moduleName"
    }
    else{
        $module
    }
}   

### <b>SQL Assessment cmdlets and SMO extension </b>
SQL Assessment API is part of the SQL Server Management Objects (SMO) July 2019 release version and higher and the SQL Server PowerShell module July 2019 release version and higher.
- [Install SMO](https://docs.microsoft.com/en-us/sql/relational-databases/server-management-objects-smo/installing-smo?view=sql-server-ver15)
- [Install SQL Server PowerShell module](https://docs.microsoft.com/en-us/sql/powershell/download-sql-server-ps-module?view=sql-server-ver15)

## Check for Azure CLI

Some notebooks also require the latest version of Azure command line interface (CLI). To install, see [https://aka.ms/installazurecliwindows](https://aka.ms/installazurecliwindows "https://aka.ms/installazurecliwindows"). Update instructions can be found at [https://aka.ms/doc/UpdateAzureCliMsi](https://aka.ms/doc/UpdateAzureCliMsi "https://aka.ms/doc/UpdateAzureCliMsi")

In [ ]:
# Check for Azure CLI
$azCli = (az --version)
if ($azCli) {        
    $azVersion = $azCli[0].Split()[-1]
    if ($azVersion -eq "*") {
        $azVersion = $azCli[0].Split()[-2]
    }
    Write-Output "Found Az CLI version... $azVersion"
}
else {
    Write-Warning "Some notebooks require the Az CLI. Please install if to run these notebooks."
}